In [18]:
import requests
import json
import re
from datetime import datetime as dt
from dateutil.parser import parse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
username = 'AQRead'
password = 'AQRead'
Server = "http://wsc.aquaticinformatics.net/AQUARIUS/publish/v2/"
server_pro = "https://wsc.aquaticinformatics.net/AQUARIUS/Provisioning/v1/"
# list of stations,
station_list = ['05BB001','05HG001']; # can be loaded from a file

In [20]:
def generateMeasurement(stationIDs, token):
    
     # for each station
    RC_df = pd.DataFrame() # initializing the date frame that holds the information of rating curves
    RC_df_temp = pd.DataFrame() # initializing the temporary data frame
    # populating the temporal data frame
    RC_df_temp = RC_df_temp.append({"Discharge": 1,\
                                    "Discharge_unit":1,\
                                    "MeanGageHeight":1,\
                                    "MeanGageHeight_unit":1,\
                                    "MeasurementStartTime":1,\
                                    "MeasurementEndTime":1,\
                                    "DischargeMethod":1,\
                                    "Station_Id":1,\
                                    "locationName":1},ignore_index=True)
    
    
    for station in stationIDs:
        # load the all field visit for one station into json file
        print (station)
        try:
            req = requests.get(Server + 'GetFieldVisitDataByLocation?LocationIdentifier='
                               + station + '&token=' + token)
            fieldDescriptions = req.json()['FieldVisitData']
        except:
            print ("Failed to load Field Visit Data for Station:" + station)
            continue

        try:
            req = requests.get(Server + 'GetLocationData?LocationIdentifier=' + station + '&token=' + token)
            locationName = req.json()['LocationName']
        except:
            print ("Failed to get station name for station:" + station)
            continue
            
        print(len(fieldDescriptions))
        
        
        for i in range(len(fieldDescriptions)):
            
            keys = fieldDescriptions[i].keys()
            
            if 'DischargeActivities' in keys:
                
                for j in range(len(fieldDescriptions[i]['DischargeActivities'])):
                    #print(fieldDescriptions[i]['DischargeActivities'][j])
                    #print(type(fieldDescriptions[i]['DischargeActivities'][j]))
                    dictionary = fieldDescriptions[i]['DischargeActivities'][j]
                    dictionary = dictionary['DischargeSummary']
                    
                    # populating the data frame
                    RC_df_temp ['Discharge'] = dictionary['Discharge']['Numeric'] # measured discharge
                    RC_df_temp ['Discharge_unit'] = dictionary['Discharge']['Unit'] # unit of dischrarge
                    RC_df_temp ['MeanGageHeight'] = dictionary['MeanGageHeight']['Numeric'] # gauge height
                    RC_df_temp ['MeanGageHeight_unit'] = dictionary['MeanGageHeight']['Unit'] # height unit
                    RC_df_temp ['MeasurementStartTime'] = dictionary['MeasurementStartTime'] # Start time
                    RC_df_temp ['MeasurementEndTime'] = dictionary['MeasurementEndTime'] # End time
                    RC_df_temp ['DischargeMethod'] = dictionary['DischargeMethod'] # Method
                    RC_df_temp ['Station_Id'] = station # name of the station
                    RC_df_temp ['locationName'] = locationName # name of the station
                    # appending to the general dataframe
                    RC_df = RC_df.append(RC_df_temp) # populate the infromation of the rating curve point
    
    # print the extracted information           
    print(RC_df)
                    
                    

            
def getToken(userName, password):
    s = requests.Session()
    data = '{"Username": "' + userName + '", "EncryptedPassword": "' + password + '", "Locale": ""}'
    url = server_pro + 'session'
    s.get(url)
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    r = s.post(url, data=data, headers=headers)
    token = r.text
    return token


def main():
    stationIDs = station_list
    token = getToken(username, password)
    print(token)
    generateMeasurement(stationIDs, token)

if __name__ == "__main__":
    main()

B0SKpCy_CEfPfk7C1PvACw2
05BB001
294
05HG001
244
    Discharge                    DischargeMethod Discharge_unit  \
0        13.5                        Mid-section          m^3/s   
0       193.0  Acoustic Doppler Current Profiler          m^3/s   
0       125.0  Acoustic Doppler Current Profiler          m^3/s   
0        18.8                        Mid-section          m^3/s   
0         6.5                        Mid-section          m^3/s   
..        ...                                ...            ...   
0       699.0                        Mid-section          m^3/s   
0       847.0                        Mid-section          m^3/s   
0      1263.0                        Mid-section          m^3/s   
0      1189.0                        Mid-section          m^3/s   
0      1467.0                        Mid-section          m^3/s   

    MeanGageHeight MeanGageHeight_unit                 MeasurementEndTime  \
0            1.904                   m  2020-10-26T08:10:00.0000000-07

In [15]:
print(RC_df)

NameError: name 'RC_df' is not defined